# MOSFET vs BJT — When to Use Which

**Module 04 — MOSFETs | Notebook 02**

---

You now know how both BJTs and MOSFETs work. The practical question is: **which one should I use?** The answer depends on your application — and it's not always obvious. This notebook gives you a clear framework for choosing.

## Concept — The Master Comparison Table

| Feature | BJT (e.g., 2N3904) | MOSFET (e.g., 2N7000/IRFZ44N) |
|---|---|---|
| **Control mechanism** | Current into base (I_B) | Voltage on gate (V_GS) |
| **Input impedance** | Low (~1-10 kΩ) | Extremely high (~10¹² Ω) |
| **DC drive current** | Continuous I_B required | Zero (gate is a capacitor) |
| **Switching power loss** | V_CE(sat) × I_C | I_D² × R_DS(on) |
| **V_CE(sat) / V_DS(on)** | ~0.1-0.3V (relatively constant) | I_D × R_DS(on) (proportional to current) |
| **Switching speed** | Limited by stored charge (storage time) | Limited by gate capacitance (Q_g) |
| **Thermal behavior** | Negative temp coefficient (thermal runaway risk!) | Positive temp coefficient (self-balancing!) |
| **Parallel operation** | Difficult (current hogging) | Easy (natural current sharing) |
| **ESD sensitivity** | Robust | Sensitive (thin gate oxide) |
| **Cost (small signal)** | Very cheap ($0.01-0.05) | Cheap ($0.02-0.10) |
| **Cost (power)** | Moderate | Low for equivalent capability |
| **Linear operation** | Excellent (amplifiers, current sources) | Good but more complex biasing |
| **On/off as switch** | Good | Excellent |

## Concept — Current-Controlled vs. Voltage-Controlled: Why It Matters

### BJT: Current-controlled
```
    Driving a BJT from a microcontroller:
    
    MCU pin ──[1kΩ]── Base
                        │
    I_B = (3.3V - 0.7V) / 1kΩ = 2.6 mA
    
    That 2.6 mA comes FROM the MCU pin.
    For β=100: I_C(max) = 260 mA
    MCU must source current continuously.
```

### MOSFET: Voltage-controlled
```
    Driving a MOSFET from a microcontroller:
    
    MCU pin ──[100Ω]── Gate
                         │
    V_GS = 3.3V (no current flows at DC!)
    
    The 100Ω only limits current during transitions
    (charging gate capacitance). At DC, the MCU
    provides zero current. The MOSFET gate is
    a capacitor — just set the voltage and walk away.
```

**Practical consequence:** With a BJT, the driver (microcontroller) must supply continuous current proportional to the load current. With a MOSFET, the driver just needs to set a voltage — the load current could be 100A and the gate current is still zero at DC.

## Concept — Where Each Device Wins

### BJT Advantages

1. **Simple biasing for analog circuits** — A single resistor gives you a usable bias point. MOSFETs need more careful biasing due to the square-law characteristic and wide V_th variation.

2. **Better linear amplification at low frequencies** — BJTs have higher transconductance (gm) per unit current: gm = I_C / V_T (where V_T ≈ 26mV). At I_C = 1mA, gm = 38 mS. A MOSFET at the same current has gm of perhaps 2-5 mS.

3. **No ESD concerns** — The base-emitter junction is robust. You can handle BJTs without worry.

4. **Low V_CE(sat) at moderate currents** — A saturated 2N3904 drops only 0.2V at 100mA, regardless of current (within limits).

5. **RF/analog applications** — BJTs (especially SiGe HBTs) dominate in low-noise amplifiers, RF mixers, and precision analog circuits.

### MOSFET Advantages

1. **No gate current** — Simplifies driver design, enables very high impedance inputs.

2. **Lower switching losses** — No minority carrier storage time. Switching is limited only by how fast you can charge/discharge the gate capacitance.

3. **R_DS(on) scaling** — Power MOSFETs can achieve incredibly low on-resistance (< 1 mΩ for modern devices).

4. **Natural parallel operation** — Positive temperature coefficient means MOSFETs share current automatically.

5. **Higher efficiency in power conversion** — Switch-mode power supplies, motor drives, and power converters almost universally use MOSFETs or IGBTs.

6. **Digital logic** — All modern digital ICs (CPUs, memory, FPGAs) are built with CMOS (complementary MOSFETs).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Power dissipation comparison: BJT vs MOSFET at different load currents

I_load = np.linspace(0.001, 10, 1000)  # Load current (A)

# BJT: 2N3904 (small signal) — V_CE(sat) model
# V_CE(sat) ≈ 0.2V at low current, rises with current
V_CE_sat_3904 = 0.1 + 0.15 * np.log1p(I_load / 0.01)  # simplified model
V_CE_sat_3904 = np.clip(V_CE_sat_3904, 0.1, 1.5)
P_BJT_3904 = V_CE_sat_3904 * I_load

# BJT: TIP31 (power BJT) — V_CE(sat) ≈ 0.4-1.0V
V_CE_sat_TIP31 = 0.3 + 0.05 * I_load
V_CE_sat_TIP31 = np.clip(V_CE_sat_TIP31, 0.3, 2.0)
P_BJT_TIP31 = V_CE_sat_TIP31 * I_load

# MOSFET: 2N7000 — R_DS(on) = 1.8 Ω
P_MOSFET_2N7000 = I_load**2 * 1.8

# MOSFET: IRFZ44N — R_DS(on) = 17.5 mΩ at V_GS=10V
P_MOSFET_IRFZ44N = I_load**2 * 0.0175

# MOSFET: Modern low-R_DS(on) — e.g., 2 mΩ
P_MOSFET_modern = I_load**2 * 0.002

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Left: Low current range (0 to 1A)
mask_low = I_load <= 1.0
ax1.plot(I_load[mask_low], P_BJT_3904[mask_low] * 1000, 'r-', linewidth=2.5, label='2N3904 (BJT)')
ax1.plot(I_load[mask_low], P_MOSFET_2N7000[mask_low] * 1000, 'g--', linewidth=2.5, label='2N7000 (MOSFET, 1.8Ω)')
ax1.plot(I_load[mask_low], P_MOSFET_IRFZ44N[mask_low] * 1000, 'b-.', linewidth=2.5, label='IRFZ44N (MOSFET, 17.5mΩ)')

# Find crossover: V_CE_sat * I = I^2 * R_DS → I = V_CE_sat / R_DS
# For 2N7000 vs 2N3904: ~0.2V / 1.8Ω ≈ 0.11A
ax1.axvline(x=0.11, color='gray', linestyle=':', alpha=0.5)
ax1.annotate('2N7000 = 2N3904\n~110 mA', xy=(0.11, 100), fontsize=9,
            ha='center', color='gray',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

ax1.set_xlabel('Load Current (A)', fontsize=12)
ax1.set_ylabel('Switch Power Loss (mW)', fontsize=12)
ax1.set_title('Low-Current Range: BJT vs Small MOSFET', fontsize=13)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 1)
ax1.set_ylim(0, 500)

# Right: High current range (0 to 10A)
ax2.plot(I_load, P_BJT_TIP31, 'r-', linewidth=2.5, label='TIP31 (Power BJT)')
ax2.plot(I_load, P_MOSFET_IRFZ44N, 'b-.', linewidth=2.5, label='IRFZ44N (17.5mΩ)')
ax2.plot(I_load, P_MOSFET_modern, 'm:', linewidth=2.5, label='Modern MOSFET (2mΩ)')

ax2.set_xlabel('Load Current (A)', fontsize=12)
ax2.set_ylabel('Switch Power Loss (W)', fontsize=12)
ax2.set_title('High-Current Range: Power BJT vs Power MOSFETs', fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 5)

plt.tight_layout()
plt.show()

print("LEFT: At very low currents (< 110mA), the 2N7000 beats the 2N3904.")
print("      But above that, the 2N7000's high R_DS(on) makes it lose badly.")
print("      The IRFZ44N wins at ALL current levels in this range.")
print("")
print("RIGHT: At high currents, the MOSFET advantage is dramatic.")
print(f"       At 10A: TIP31 dissipates ~{P_BJT_TIP31[-1]:.1f}W, IRFZ44N only {P_MOSFET_IRFZ44N[-1]:.2f}W")
print(f"       That's {P_BJT_TIP31[-1]/P_MOSFET_IRFZ44N[-1]:.0f}x less power in the switch!")

In [ ]:
# Plot the crossover current where MOSFET becomes more efficient than BJT
# Crossover: V_CE(sat) * I = I^2 * R_DS(on) → I_cross = V_CE(sat) / R_DS(on)

import numpy as np
import matplotlib.pyplot as plt

R_DS_values = np.logspace(-3, 1, 200)  # 1 mΩ to 10 Ω
V_CE_sat_values = [0.1, 0.2, 0.3, 0.5, 1.0]  # Different BJT V_CE(sat)

fig, ax = plt.subplots(figsize=(10, 6))

colors = plt.cm.Reds(np.linspace(0.3, 0.9, len(V_CE_sat_values)))

for V_CE, color in zip(V_CE_sat_values, colors):
    I_crossover = V_CE / R_DS_values
    ax.loglog(R_DS_values * 1000, I_crossover, linewidth=2, color=color,
             label=f'V_CE(sat) = {V_CE} V')

# Mark some common MOSFETs
mosfets = {
    '2N7000': 1800,      # 1.8 Ω → mΩ
    'IRFZ44N': 17.5,     # 17.5 mΩ
    'IRF3205': 8.0,      # 8 mΩ
    'Modern\n(2mΩ)': 2.0,
}

for name, rds_mohm in mosfets.items():
    ax.axvline(x=rds_mohm, color='blue', linestyle=':', alpha=0.4)
    ax.text(rds_mohm, 0.003, name, fontsize=9, ha='center', color='blue',
           rotation=90, va='bottom')

ax.fill_between([0.1, 10000], [0.001, 0.001], [100, 100], alpha=0.05, color='green')
ax.text(1, 0.005, 'MOSFET wins\n(below line)', fontsize=12, color='green', ha='center')
ax.text(1, 50, 'BJT wins\n(above line)', fontsize=12, color='red', ha='center')

ax.set_xlabel('MOSFET R_DS(on) (mΩ)', fontsize=12)
ax.set_ylabel('Crossover Current (A)', fontsize=12)
ax.set_title('BJT vs MOSFET Crossover: Below the Line, MOSFET is More Efficient', fontsize=13)
ax.legend(fontsize=10, title='BJT V_CE(sat)')
ax.grid(True, alpha=0.3, which='both')
ax.set_xlim(1, 5000)
ax.set_ylim(0.001, 100)

plt.tight_layout()
plt.show()

print("How to read this chart:")
print("  Pick your MOSFET's R_DS(on) on the x-axis.")
print("  Pick your BJT's V_CE(sat) curve.")
print("  If your load current is BELOW the line → MOSFET wins on conduction loss.")
print("  If ABOVE → BJT wins on conduction loss.")
print("")
print("Example: IRFZ44N (17.5 mΩ) vs BJT with V_CE(sat) = 0.3V")
print(f"  Crossover at {0.3/0.0175:.0f}A — MOSFET wins below {0.3/0.0175:.0f}A")
print("  Since the IRFZ44N is rated for 49A, it wins in its entire operating range!")

## Concept — Switching Speed: Storage Time vs. Gate Charge

### BJT switching limitation: Stored charge

When a BJT is in saturation (fully on as a switch), minority carriers flood the base region. To turn it OFF, you must first *remove* these stored carriers. This takes time — called the **storage time (t_s)**, typically 200-500 ns for common switching BJTs.

Sequence: Turn-off command → Wait for stored charge to drain (t_s) → Current starts falling (t_f) → OFF

### MOSFET switching limitation: Gate capacitance

A MOSFET has no stored minority carriers (it's a majority-carrier device). The switching speed is limited only by how fast you can charge/discharge the gate capacitance — specifically, the **Miller capacitance (C_GD)** during the Miller plateau.

Sequence: Turn-off command → Discharge gate capacitor → Current falls → OFF

The MOSFET wins because:
1. No storage time delay
2. Gate charge can be supplied very quickly with a strong driver
3. Modern gate drivers can switch in 10-50 ns

In [ ]:
# Switching waveform comparison: BJT vs MOSFET

import numpy as np
import matplotlib.pyplot as plt

t_ns = np.linspace(0, 500, 2000)  # Time in nanoseconds

fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# --- BJT Turn-off ---
ax = axes[0, 0]
# Input: base drive goes from high to low at t=50ns
base_drive = np.where(t_ns < 50, 1.0, 0.0)
ax.plot(t_ns, base_drive, 'b-', linewidth=2, label='Base drive')
ax.set_title('BJT Turn-Off: Base Drive Signal', fontsize=11)
ax.set_ylabel('Base Drive', fontsize=10)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 500)

ax = axes[1, 0]
# BJT collector current with storage time
t_storage = 200  # ns — stored charge must be removed
t_fall = 80      # ns — current fall time
bjt_ic = np.where(t_ns < 50, 1.0,
         np.where(t_ns < 50 + t_storage, 1.0,  # Storage time — current stays HIGH!
         np.where(t_ns < 50 + t_storage + t_fall,
                  1.0 - (t_ns - 50 - t_storage) / t_fall, 0.0)))
bjt_ic = np.clip(bjt_ic, 0, 1)

ax.plot(t_ns, bjt_ic, 'r-', linewidth=2.5, label='I_C (BJT)')
ax.axvspan(50, 50 + t_storage, alpha=0.15, color='red')
ax.annotate(f'Storage time\nt_s = {t_storage} ns', xy=(150, 0.5), fontsize=10,
           ha='center', color='red', fontweight='bold',
           bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
ax.annotate(f't_f = {t_fall} ns', xy=(50 + t_storage + t_fall/2, 0.3), fontsize=10,
           ha='center', color='darkred')
ax.set_title('BJT Turn-Off: Collector Current', fontsize=11)
ax.set_xlabel('Time (ns)', fontsize=10)
ax.set_ylabel('I_C (normalized)', fontsize=10)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 500)

# --- MOSFET Turn-off ---
ax = axes[0, 1]
gate_drive = np.where(t_ns < 50, 1.0,
             np.where(t_ns < 80, 1.0 - (t_ns - 50) / 30, 0.0))  # 30ns to discharge gate
gate_drive = np.clip(gate_drive, 0, 1)
ax.plot(t_ns, gate_drive, 'b-', linewidth=2, label='Gate drive (V_GS)')
ax.set_title('MOSFET Turn-Off: Gate Drive Signal', fontsize=11)
ax.set_ylabel('Gate Drive', fontsize=10)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 500)

ax = axes[1, 1]
# MOSFET drain current — follows gate with minimal delay
t_delay_off = 20   # ns — propagation delay
t_fall_mos = 30    # ns — current fall time
mos_id = np.where(t_ns < 50 + t_delay_off, 1.0,
         np.where(t_ns < 50 + t_delay_off + t_fall_mos,
                  1.0 - (t_ns - 50 - t_delay_off) / t_fall_mos, 0.0))
mos_id = np.clip(mos_id, 0, 1)

ax.plot(t_ns, mos_id, 'g-', linewidth=2.5, label='I_D (MOSFET)')
ax.annotate(f'No storage time!\nt_d(off)={t_delay_off}ns, t_f={t_fall_mos}ns',
           xy=(100, 0.5), fontsize=10, ha='center', color='green', fontweight='bold',
           bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
ax.set_title('MOSFET Turn-Off: Drain Current', fontsize=11)
ax.set_xlabel('Time (ns)', fontsize=10)
ax.set_ylabel('I_D (normalized)', fontsize=10)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 500)

plt.suptitle('Switching Speed Comparison: BJT vs MOSFET Turn-Off', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

bjt_total = t_storage + t_fall
mos_total = t_delay_off + t_fall_mos
print(f"BJT total turn-off time: {bjt_total} ns (storage {t_storage} + fall {t_fall})")
print(f"MOSFET total turn-off time: {mos_total} ns (delay {t_delay_off} + fall {t_fall_mos})")
print(f"MOSFET is {bjt_total/mos_total:.1f}x faster!")
print("\nThe storage time is the BJT's Achilles' heel for fast switching.")

## The Material Science Why

### Why the BJT has storage time and the MOSFET doesn't

**BJT:** The base region is flooded with *minority carriers* (electrons in PNP, holes in NPN) when saturated. These carriers have a finite **recombination lifetime** — they don't disappear instantly when you remove the base drive. Until enough carriers recombine or are swept out, the collector current keeps flowing. This is the storage time.

**MOSFET:** The channel is made of *majority carriers* (electrons in N-channel) that are held in place by the gate electric field. When you remove the gate voltage, the field disappears and the electrons simply drift away — there's no excess minority charge to recombine. The channel vanishes almost as fast as you can discharge the gate capacitor.

### Why this matters for power electronics

In a power converter switching at 100 kHz (period = 10 μs):
- BJT storage time of 200 ns = 2% of the period wasted in transitions
- MOSFET can switch in 50 ns = 0.5% of the period

During transitions, both V and I are large simultaneously → high power loss. Faster switching = less transition loss = higher efficiency.

At 1 MHz switching (modern power converters), the BJT's storage time would consume 20% of the period — completely impractical.

In [ ]:
# Decision flowchart visualization

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

fig, ax = plt.subplots(figsize=(12, 10))
ax.set_xlim(0, 12)
ax.set_ylim(0, 10)
ax.axis('off')
ax.set_title('BJT vs MOSFET Decision Flowchart', fontsize=16, fontweight='bold', pad=20)

def draw_box(ax, x, y, text, color='lightyellow', width=2.5, height=0.7):
    box = mpatches.FancyBboxPatch((x - width/2, y - height/2), width, height,
                                   boxstyle='round,pad=0.15', facecolor=color,
                                   edgecolor='black', linewidth=1.5)
    ax.add_patch(box)
    ax.text(x, y, text, ha='center', va='center', fontsize=9, fontweight='bold')

def draw_diamond(ax, x, y, text, width=3.0, height=0.8):
    diamond = plt.Polygon([(x, y + height/2), (x + width/2, y),
                            (x, y - height/2), (x - width/2, y)],
                           facecolor='lightyellow', edgecolor='black', linewidth=1.5)
    ax.add_patch(diamond)
    ax.text(x, y, text, ha='center', va='center', fontsize=8)

def draw_arrow(ax, x1, y1, x2, y2, label=''):
    ax.annotate('', xy=(x2, y2), xytext=(x1, y1),
               arrowprops=dict(arrowstyle='->', color='black', lw=1.5))
    if label:
        mx, my = (x1+x2)/2, (y1+y2)/2
        ax.text(mx + 0.15, my + 0.1, label, fontsize=8, color='blue')

# Start
draw_box(ax, 6, 9.3, 'What are you building?', 'lightblue', 3.0)

# Level 1: Power switching?
draw_diamond(ax, 3, 8.0, 'Power switching?\n(> 1A or > 1W)')
draw_arrow(ax, 6, 8.9, 3, 8.4)

draw_diamond(ax, 9, 8.0, 'High frequency?\n(> 10 kHz switching)')
draw_arrow(ax, 6, 8.9, 9, 8.4)

# Power switching → MOSFET
draw_box(ax, 1.5, 6.8, 'Use MOSFET\n(IRFZ44N, IRF3205...)', 'lightgreen', 2.8, 0.8)
draw_arrow(ax, 3, 7.6, 1.5, 7.2, 'Yes')

# Not power switching
draw_diamond(ax, 4.5, 6.5, 'Analog/linear\namplification?')
draw_arrow(ax, 3, 7.6, 4.5, 6.9, 'No')

# Analog → BJT usually
draw_box(ax, 2.5, 5.3, 'Consider BJT\n(simpler biasing,\nhigher gm)', 'lightsalmon', 2.5, 0.9)
draw_arrow(ax, 4.5, 6.1, 2.5, 5.8, 'Yes')

# High frequency → MOSFET  
draw_box(ax, 10, 6.8, 'Use MOSFET\n(no storage time)', 'lightgreen', 2.5, 0.8)
draw_arrow(ax, 9, 7.6, 10, 7.2, 'Yes')

# Not high freq
draw_diamond(ax, 7.5, 6.5, 'MCU/logic\ndriven?')
draw_arrow(ax, 9, 7.6, 7.5, 6.9, 'No')

# MCU driven → Logic-level MOSFET
draw_box(ax, 9.5, 5.3, 'Logic-level MOSFET\n(IRLZ44N, 2N7000)', 'lightgreen', 2.8, 0.8)
draw_arrow(ax, 7.5, 6.1, 9.5, 5.7, 'Yes')

# Not MCU driven → simple low-current?
draw_diamond(ax, 5.5, 5.0, 'Simple, low-cost,\nlow current?')
draw_arrow(ax, 7.5, 6.1, 5.5, 5.4, 'No')

draw_box(ax, 4, 3.8, 'BJT is fine\n(2N3904/2N3906)', 'lightsalmon', 2.5, 0.8)
draw_arrow(ax, 5.5, 4.6, 4, 4.2, 'Yes')

draw_box(ax, 7.5, 3.8, 'Either works —\nchoose MOSFET\nfor efficiency', 'lightyellow', 2.8, 0.9)
draw_arrow(ax, 5.5, 4.6, 7.5, 4.3, 'No')

# Summary box
summary = ('Rule of thumb: When in doubt, use a MOSFET.\n'
           'BJTs still shine for analog, RF, and dead-simple circuits.')
ax.text(6, 2.5, summary, ha='center', va='center', fontsize=11,
       style='italic', color='navy',
       bbox=dict(boxstyle='round,pad=0.5', facecolor='aliceblue', edgecolor='navy'))

plt.tight_layout()
plt.show()

## Concept — Power Dissipation: V_CE(sat) × I_C vs. I_D² × R_DS(on)

This is the fundamental difference in how the two devices waste power when used as switches:

### BJT: P = V_CE(sat) × I_C
- V_CE(sat) is roughly **constant** (~0.1-0.3V) regardless of current (within the device's range)
- Power dissipation scales **linearly** with current
- At low currents: small but nonzero (always paying the V_CE(sat) tax)
- At high currents: grows linearly

### MOSFET: P = I_D² × R_DS(on)
- R_DS(on) is roughly **constant** at a given V_GS and temperature
- Power dissipation scales with the **square** of current
- At low currents: extremely small (I² approaches zero fast!)
- At high currents: grows quadratically — eventually catches up to and exceeds the BJT

**The crossover point:** I_cross = V_CE(sat) / R_DS(on)

For most power MOSFETs, this crossover is well above the device's maximum rated current, so the MOSFET wins throughout its operating range. For small-signal MOSFETs like the 2N7000 (high R_DS(on)), the crossover happens at a low current.

## Experiment 1 — Same LED, Different Switch: BJT vs MOSFET

**Goal:** Build the same LED switching circuit with a 2N3904 (BJT) and a 2N7000 (MOSFET), and compare voltage drops and input current.

### Circuit A: BJT (2N3904)

```
    +5V
     │
    [330Ω]
     │
    [LED]
     │
     └──── Collector (2N3904)
                │
    +5V──[10kΩ]─┤ Base
                │
               Emitter ── GND
```

### Circuit B: MOSFET (2N7000)

```
    +5V
     │
    [330Ω]
     │
    [LED]
     │
     └──── Drain (2N7000)
                │
    +5V──[10kΩ]─┤ Gate
                │
              Source ── GND
```

### Measurements

| Parameter | BJT (2N3904) | MOSFET (2N7000) |
|---|---|---|
| V_drop across switch (V_CE / V_DS) | _____ mV | _____ mV |
| LED current (I_C / I_D) | _____ mA | _____ mA |
| Input current (I_B / I_G) | _____ mA | _____ mA |
| Input voltage (V_BE / V_GS) | _____ V | _____ V |
| Switch power loss | _____ mW | _____ mW |

### Expected Results

- **V_CE(sat) ≈ 0.1-0.3V** vs **V_DS ≈ 0.02-0.1V** — MOSFET wins slightly
- **I_B ≈ 0.43 mA** (= (5V - 0.7V) / 10kΩ) vs **I_G ≈ 0 mA** — MOSFET wins dramatically
- At this low current (~10 mA), the power difference is negligible, but the principle is clear

## Experiment 2 — Measuring Base Current vs. Gate Current

**Goal:** Directly measure and compare the input current for both devices.

### Procedure

**BJT (2N3904):**
1. Build Circuit A from Experiment 1
2. Put DMM in **current mode** in series with the 10kΩ base resistor
3. Measure I_B: should be approximately (5V - 0.7V) / 10kΩ ≈ **0.43 mA**

**MOSFET (2N7000):**
1. Build Circuit B from Experiment 1
2. Put DMM in **current mode** in series with the 10kΩ gate resistor
3. Measure I_G: should read **0.00 mA** (below DMM resolution)

### Why This Matters

The BJT's 0.43 mA of base current is "wasted" — it does no useful work in the load. It comes from the driving circuit (here, the 5V supply through the resistor). In a microcontroller-driven circuit, this current comes from the MCU's I/O pin, reducing the available current for other pins.

The MOSFET's zero gate current means:
- The driving circuit does zero DC work
- You can drive many MOSFET gates in parallel without loading the source
- The driver only needs to supply transient current during switching (charging the gate capacitor)

In [ ]:
# Thermal behavior comparison: BJT vs MOSFET

import numpy as np
import matplotlib.pyplot as plt

T = np.linspace(25, 150, 200)  # Temperature in °C

# BJT: V_CE(sat) DECREASES with temperature (negative temp coefficient)
# This means: hotter BJT → lower V_CE → carries MORE current → gets HOTTER → thermal runaway!
V_CE_sat = 0.3 * (1 - 0.002 * (T - 25))  # Simplified model

# MOSFET: R_DS(on) INCREASES with temperature (positive temp coefficient)  
# This means: hotter MOSFET → higher resistance → carries LESS current → cools down → stable!
R_DS_on = 17.5e-3 * (1 + 0.005 * (T - 25))  # Simplified model, ~+0.5%/°C

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# BJT: V_CE(sat) vs temperature
ax1.plot(T, V_CE_sat * 1000, 'r-', linewidth=2.5)
ax1.set_xlabel('Junction Temperature (°C)', fontsize=12)
ax1.set_ylabel('V_CE(sat) (mV)', fontsize=12)
ax1.set_title('BJT: V_CE(sat) vs Temperature', fontsize=13)
ax1.annotate('V_CE(sat) DECREASES\n→ More current flows\n→ Gets HOTTER\n→ THERMAL RUNAWAY!',
            xy=(100, 240), fontsize=11, color='red',
            bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
ax1.grid(True, alpha=0.3)

# MOSFET: R_DS(on) vs temperature  
ax2.plot(T, R_DS_on * 1000, 'g-', linewidth=2.5)
ax2.set_xlabel('Junction Temperature (°C)', fontsize=12)
ax2.set_ylabel('R_DS(on) (mΩ)', fontsize=12)
ax2.set_title('MOSFET: R_DS(on) vs Temperature', fontsize=13)
ax2.annotate('R_DS(on) INCREASES\n→ Less current flows\n→ Cools DOWN\n→ SELF-STABILIZING!',
            xy=(80, 22), fontsize=11, color='green',
            bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("This is why you can safely parallel MOSFETs but not BJTs (without extra circuitry).")
print(f"")
print(f"IRFZ44N R_DS(on): {17.5:.1f} mΩ at 25°C → {17.5 * (1 + 0.005 * 100):.1f} mΩ at 125°C")
print(f"That's a {(0.005 * 100) * 100:.0f}% increase — significant for thermal design!")

## Concept — The Real World: Where Each Lives Today

### MOSFETs dominate:
- **Digital logic** — every CPU, GPU, memory chip uses billions of MOSFETs (CMOS)
- **Power conversion** — switch-mode supplies, motor drives, solar inverters
- **Power switching** — LED drivers, relay replacements, load switches
- **Automotive** — smart power switches, body control modules

### BJTs still thrive:
- **Low-noise analog** — precision op-amp input stages (bipolar op-amps like OP27)
- **RF/microwave** — SiGe HBTs for 5G, radar, communications (f_T > 300 GHz!)
- **Voltage references** — bandgap references use BJT characteristics
- **Current mirrors/sources** — BJTs are naturally suited for this
- **Simple hobby circuits** — when you just need to switch an LED and have 2N3904s on hand

### IGBTs: The hybrid
The IGBT (Insulated Gate Bipolar Transistor) combines a MOSFET gate with a BJT output stage. It's used for very high power applications (hundreds of amps, thousands of volts) like electric vehicle motor drives and industrial inverters.

## Simulation — Build It in Falstad

Compare BJT and MOSFET switching side by side.

**What to build:**

1. Open [Falstad (blank canvas)](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKCA)
2. Build two identical LED circuits side by side:
   - **Left side:** NPN BJT with a 10 kΩ base resistor, LED + 330Ω from 5V to collector, emitter to ground
   - **Right side:** N-channel MOSFET with a 10 kΩ gate resistor, LED + 330Ω from 5V to drain, source to ground
3. Connect both inputs (base resistor and gate resistor) to the same switch or voltage source

**What to observe:**
- Both LEDs turn on and off together — from the outside, they do the same job
- Hover over the BJT base resistor: current flows through it (the BJT needs base current to stay on)
- Hover over the MOSFET gate resistor: no steady-state current (the MOSFET gate is capacitive — current only flows briefly during switching)
- This is the key tradeoff: BJTs need continuous input current; MOSFETs need only a voltage

**What to think about:**
- In a circuit that switches thousands of times per second, that gate resistor current during each transition adds up — this is why gate drive design matters for high-frequency MOSFET switching
- For a simple on/off switch, the MOSFET wins on input power. For analog amplification at low voltages, the BJT's transconductance advantage may win.

## Datasheet Connection

When choosing between BJT and MOSFET for a design, compare these datasheet parameters:

| Design Question | BJT Parameter | MOSFET Parameter |
|---|---|---|
| How much power does the switch waste? | V_CE(sat) × I_C | I_D² × R_DS(on) |
| How much input drive do I need? | I_B = I_C / h_FE | Q_g (gate charge) |
| How fast can I switch? | t_s + t_f (storage + fall) | t_d(off) + t_f |
| Can my MCU drive it directly? | Need enough I_B from pin | Need V_GS > V_GS(th) from pin |
| What happens when it gets hot? | V_CE(sat) drops (runaway risk) | R_DS(on) rises (self-limiting) |
| Can I parallel them? | Not easily | Yes, naturally |

The IRFZ44N datasheet has a curve showing R_DS(on) vs. temperature — look at it! You'll see R_DS(on) nearly doubles from 25°C to 125°C.

## Checkpoint Questions

1. **You need to switch a 12V, 5A DC motor from a 3.3V microcontroller.** BJT or MOSFET? Which specific device from our kit would you choose, and why?

2. **You're building a precision current source for a sensor.** Would you lean toward BJT or MOSFET? Why?

3. **Explain in physical terms** why a BJT has storage time but a MOSFET doesn't.

4. **You have two identical MOSFETs in parallel, and one gets hotter.** What happens to the current sharing? Now answer the same for two BJTs.

5. **Calculate:** A MOSFET with R_DS(on) = 50 mΩ is switching 3A. A BJT with V_CE(sat) = 0.2V is switching the same 3A. Which dissipates more power in the switch?

6. **You're driving 20 LEDs, each needing its own switch, from a microcontroller with limited I/O current.** BJT or MOSFET for each switch? Why?

7. **Why is the IGBT used instead of a MOSFET at very high voltages (> 600V)?**

---

*Answers:*
1. *MOSFET. At 5A, MOSFET R_DS(on) losses are much lower. Use a logic-level MOSFET since the MCU outputs 3.3V. The 2N7000 can't handle 5A. The IRFZ44N needs 10V gate drive. You'd need a logic-level power MOSFET like IRLZ44N, or use the 2N7000/BJT as a gate driver for the IRFZ44N.*
2. *BJT — its exponential I-V relationship (I_C = I_S × e^(V_BE/V_T)) and high transconductance make it natural for precision current sources. The V_BE is well-defined and predictable.*
3. *BJT: minority carriers injected into the base during saturation must recombine or be swept out — takes time. MOSFET: majority carrier channel is maintained by electric field; remove field and channel disappears (no stored charge to remove).*
4. *MOSFET: hotter device has higher R_DS(on), carries less current, tends to cool — stable. BJT: hotter device has lower V_BE, draws more current, gets even hotter — thermal runaway.*
5. *MOSFET: P = 3² × 0.05 = 0.45W. BJT: P = 0.2 × 3 = 0.6W. MOSFET dissipates 25% less.*
6. *MOSFET — each gate draws zero DC current, so 20 gates don't load the MCU at all. 20 BJT bases would need 20 × I_B of continuous current from the MCU.*
7. *At high voltages, achieving low R_DS(on) requires very thick, lightly-doped drift regions → R_DS(on) becomes impractically high. The IGBT's bipolar output stage uses conductivity modulation to maintain low on-voltage even at high voltage ratings.*